In [1]:
import sqlite3

conn = sqlite3.connect("parks_and_recreation.db")
cursor = conn.cursor()

sql_script = """
DROP TABLE IF EXISTS employee_demographics;
DROP TABLE IF EXISTS employee_salary;
DROP TABLE IF EXISTS parks_departments;

CREATE TABLE employee_demographics (
  employee_id INTEGER PRIMARY KEY,
  first_name TEXT,
  last_name TEXT,
  age INTEGER,
  gender TEXT,
  birth_date TEXT
);

CREATE TABLE employee_salary (
  employee_id INTEGER NOT NULL,
  first_name TEXT NOT NULL,
  last_name TEXT NOT NULL,
  occupation TEXT,
  salary INTEGER,
  dept_id INTEGER
);

INSERT INTO employee_demographics (employee_id, first_name, last_name, age, gender, birth_date)
VALUES
(1,'Leslie','Knope',44,'Female','1979-09-25'),
(3,'Tom','Haverford',36,'Male','1987-03-04'),
(4,'April','Ludgate',29,'Female','1994-03-27'),
(5,'Jerry','Gergich',61,'Male','1962-08-28'),
(6,'Donna','Meagle',46,'Female','1977-07-30'),
(7,'Ann','Perkins',35,'Female','1988-12-01'),
(8,'Chris','Traeger',43,'Male','1980-11-11'),
(9,'Ben','Wyatt',38,'Male','1985-07-26'),
(10,'Andy','Dwyer',34,'Male','1989-03-25'),
(11,'Mark','Brendanawicz',40,'Male','1983-06-14'),
(12,'Craig','Middlebrooks',37,'Male','1986-07-27');

INSERT INTO employee_salary (employee_id, first_name, last_name, occupation, salary, dept_id)
VALUES
(1,'Leslie','Knope','Deputy Director of Parks and Recreation',75000,1),
(2,'Ron','Swanson','Director of Parks and Recreation',70000,1),
(3,'Tom','Haverford','Entrepreneur',50000,1),
(4,'April','Ludgate','Assistant to the Director of Parks and Recreation',25000,1),
(5,'Jerry','Gergich','Office Manager',50000,1),
(6,'Donna','Meagle','Office Manager',60000,1),
(7,'Ann','Perkins','Nurse',55000,4),
(8,'Chris','Traeger','City Manager',90000,3),
(9,'Ben','Wyatt','State Auditor',70000,6),
(10,'Andy','Dwyer','Shoe Shiner and Musician',20000,NULL),
(11,'Mark','Brendanawicz','City Planner',57000,3),
(12,'Craig','Middlebrooks','Parks Director',65000,1);

CREATE TABLE parks_departments (
  department_id INTEGER PRIMARY KEY AUTOINCREMENT,
  department_name TEXT NOT NULL
);

INSERT INTO parks_departments (department_name)
VALUES
('Parks and Recreation'),
('Animal Control'),
('Public Works'),
('Healthcare'),
('Library'),
('Finance');
"""

cursor.executescript(sql_script)
conn.commit()
conn.close()

print("SQLite database created successfully.")


SQLite database created successfully.


See what tables the database has

In [29]:
conn = sqlite3.connect("parks_and_recreation.db")
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")

# Fetch all the results
results = cursor.fetchall()  # This returns a list of tuples
print("Raw results:", results)

# Create an empty list to store table names
tables = []

# Loop over each row in the results
for row in results:
    table_name = row[0]   # Take the first element of the tuple
    tables.append(table_name)  # Add it to the list

# Now you have a clean list of table names
print("Tables found:", tables)


Raw results: [('employee_demographics',), ('employee_salary',), ('parks_departments',), ('sqlite_sequence',)]
Tables found: ['employee_demographics', 'employee_salary', 'parks_departments', 'sqlite_sequence']


Read the tables into dataframes

In [30]:
dfs = {} # Dictionary to hold DataFrames

for table in tables:
    dfs[table] = pd.read_sql_query(f"SELECT * FROM {table}", conn)

print(dfs["employee_salary"].head())
print(dfs["employee_demographics"].head())

df1 = dfs["employee_salary"]
df2 = dfs["employee_demographics"]


   employee_id first_name  last_name  \
0            1     Leslie      Knope   
1            2        Ron    Swanson   
2            3        Tom  Haverford   
3            4      April    Ludgate   
4            5      Jerry    Gergich   

                                          occupation  salary  dept_id  
0            Deputy Director of Parks and Recreation   75000      1.0  
1                   Director of Parks and Recreation   70000      1.0  
2                                       Entrepreneur   50000      1.0  
3  Assistant to the Director of Parks and Recreation   25000      1.0  
4                                     Office Manager   50000      1.0  
   employee_id first_name  last_name  age  gender  birth_date
0            1     Leslie      Knope   44  Female  1979-09-25
1            3        Tom  Haverford   36    Male  1987-03-04
2            4      April    Ludgate   29  Female  1994-03-27
3            5      Jerry    Gergich   61    Male  1962-08-28
4            6   

In [ ]:
df2['gender'].unique()

'Female'

In [44]:
df2[df2['age'] > 40]

,employee_id,first_name,last_name,age,gender,birth_date
0,1,Leslie,Knope,44,Female,1979-09-25
3,5,Jerry,Gergich,61,Male,1962-08-28
4,6,Donna,Meagle,46,Female,1977-07-30
6,8,Chris,Traeger,43,Male,1980-11-11


In [45]:
df2[df2['first_name'] == 'Leslie']

,employee_id,first_name,last_name,age,gender,birth_date
0,1,Leslie,Knope,44,Female,1979-09-25


In [54]:
df2[(df2['age'] > 30) & (df2['gender'] == 'Male')]

,employee_id,first_name,last_name,age,gender,birth_date
1,3,Tom,Haverford,36,Male,1987-03-04
3,5,Jerry,Gergich,61,Male,1962-08-28
6,8,Chris,Traeger,43,Male,1980-11-11
7,9,Ben,Wyatt,38,Male,1985-07-26
8,10,Andy,Dwyer,34,Male,1989-03-25
9,11,Mark,Brendanawicz,40,Male,1983-06-14
10,12,Craig,Middlebrooks,37,Male,1986-07-27


In [56]:
df2[df2['first_name'].str.contains('an', case=False)]

,employee_id,first_name,last_name,age,gender,birth_date
5,7,Ann,Perkins,35,Female,1988-12-01
8,10,Andy,Dwyer,34,Male,1989-03-25


In [59]:
result = df1.groupby('occupation').agg(
    avg_salary=('salary', 'mean'),
    min_salary=('salary', 'min'),
    max_salary=('salary', 'max'),
    count_occupation=('occupation', 'count')
).reset_index()

print(result)


                                           occupation  avg_salary  min_salary  \
0   Assistant to the Director of Parks and Recreation     25000.0       25000   
1                                        City Manager     90000.0       90000   
2                                        City Planner     57000.0       57000   
3             Deputy Director of Parks and Recreation     75000.0       75000   
4                    Director of Parks and Recreation     70000.0       70000   
5                                        Entrepreneur     50000.0       50000   
6                                               Nurse     55000.0       55000   
7                                      Office Manager     55000.0       50000   
8                                      Parks Director     65000.0       65000   
9                            Shoe Shiner and Musician     20000.0       20000   
10                                      State Auditor     70000.0       70000   

    max_salary  count_occup

In [63]:
result = df2.sort_values(
    by=['gender', 'age'],        # first sort by gender, then by age
    ascending=[True, False]      # gender ASC, age DESC
)

print(result)


    employee_id first_name     last_name  age  gender  birth_date
4             6      Donna        Meagle   46  Female  1977-07-30
0             1     Leslie         Knope   44  Female  1979-09-25
5             7        Ann       Perkins   35  Female  1988-12-01
2             4      April       Ludgate   29  Female  1994-03-27
3             5      Jerry       Gergich   61    Male  1962-08-28
6             8      Chris       Traeger   43    Male  1980-11-11
9            11       Mark  Brendanawicz   40    Male  1983-06-14
7             9        Ben         Wyatt   38    Male  1985-07-26
10           12      Craig  Middlebrooks   37    Male  1986-07-27
1             3        Tom     Haverford   36    Male  1987-03-04
8            10       Andy         Dwyer   34    Male  1989-03-25


In [64]:
result = pd.merge(
    df2[['employee_id', 'gender']],  # select columns from demographics
    df1[['employee_id', 'salary']],        # select columns from salary
    on='employee_id',                            # join on employee_id
    how='inner'                                  # inner join (default)
)

print(result)


    employee_id  gender  salary
0             1  Female   75000
1             3    Male   50000
2             4  Female   25000
3             5    Male   50000
4             6  Female   60000
5             7  Female   55000
6             8    Male   90000
7             9    Male   70000
8            10    Male   20000
9            11    Male   57000
10           12    Male   65000
